In [3]:
import json

path = "data/task_info.json"
with open(path, "r") as f:
    data = json.load(f)

print(data)
# {
#     "obtain_planks": {
#       "question": "How to obtain planks?",
#       "group": "MT1",
#       "alias": "basic",
#       "episode": 3000,
#       "object": "planks"
#     },
#     ...
# }


# filter to get the keys of the group "MT1"

group = "MT1"
keys = [k for k, v in data.items() if v["group"] == group]
keys

{'obtain_planks': {'question': 'How to obtain planks?', 'group': 'MT1', 'alias': 'basic', 'episode': 3000, 'object': 'planks'}, 'obtain_stick': {'question': 'How to obtain stick?', 'group': 'MT1', 'alias': 'basic', 'episode': 3000, 'object': 'stick'}, 'obtain_wooden_slab': {'question': 'How to obtain wooden_slab?', 'group': 'MT1', 'alias': 'basic', 'episode': 3000, 'object': 'wooden_slab'}, 'obtain_wooden_button': {'question': 'How to obtain wooden_button?', 'group': 'MT1', 'alias': 'basic', 'episode': 3000, 'object': 'wooden_button'}, 'obtain_wooden_pressure_plate': {'question': 'How to obtain wooden_pressure_plate?', 'group': 'MT1', 'alias': 'basic', 'episode': 3000, 'object': 'wooden_pressure_plate'}, 'obtain_chest': {'question': 'How to obtain chest?', 'group': 'MT1', 'alias': 'basic', 'episode': 3000, 'object': 'chest'}, 'obtain_oak_stairs': {'question': 'How to obtain oak_stairs?', 'group': 'MT1', 'alias': 'basic', 'episode': 3000, 'object': 'oak_stairs'}, 'obtain_sign': {'questi

['obtain_planks',
 'obtain_stick',
 'obtain_wooden_slab',
 'obtain_wooden_button',
 'obtain_wooden_pressure_plate',
 'obtain_chest',
 'obtain_oak_stairs',
 'obtain_sign',
 'obtain_fence',
 'obtain_fence_gate',
 'obtain_boat',
 'obtain_trapdoor',
 'obtain_bowl']

In [19]:
import json
import math

tech_tree = {}

with open("data/goal_lib.json", "r") as f:
    goal_lib = json.load(f)
for g in goal_lib:
    k = g.replace("smelt_", "").replace("craft_", "").replace("mine_", "")
    tech_tree[k] = goal_lib[g]
    tech_tree[k]["name"] = k


# tech_tree
# class Node:
#     def __init__(self, name, quantity_needed):
#         self.name = name
#         self.quantity_needed = quantity_needed
#         self.children = []
#         self.visited = False

#     def __repr__(self) -> str:
#         return f"{self.name} {self.quantity_needed} {self.children}"


def get_plan(target, need=1) -> list[dict]:
    goal = tech_tree[target]
    goal["quantity_needed"] = need
    goal["depth"] = 0
    tree = {target: goal}

    def travel_tech_tree(current: str, quantity_needed: int, depth=1):
        """
        Recursive function to travel the tech tree
        """

        # add children
        requirements = tech_tree[current]["precondition"] | tech_tree[current]["tool"]
        quantity_to_produce = tech_tree[current]["output"][current]
        for r in requirements:
            cost_to_produce = requirements[r]
            if quantity_to_produce < quantity_needed:
                cost_to_produce = math.ceil(
                    cost_to_produce * (quantity_needed / quantity_to_produce)
                )
            # node already exists
            if r in tree:
                tree[r]["quantity_needed"] += cost_to_produce
                tree[r]["depth"] = max(tree[r]["depth"], depth)
                travel_tech_tree(r, cost_to_produce, depth=depth + 1)

            # new tech
            else:
                tree[r] = tech_tree[r]
                tree[r]["quantity_needed"] = cost_to_produce
                tree[r]["depth"] = depth
                travel_tech_tree(r, cost_to_produce, depth=depth + 1)

    travel_tech_tree(target, need)

    # sort by depth
    plan = sorted(tree.values(), key=lambda x: x["depth"], reverse=True)
    return plan


for q in data:
    question = data[q]["question"]
    target = question.split()[-1].replace("?", "")
    # print(target)
    get_plan(target)


target = "wooden_slab"
p = get_plan(target)
# print(p)
print([(x["name"], x["quantity_needed"], x["depth"]) for x in p])

[('log', 2, 3), ('planks', 7, 2), ('crafting_table', 1, 1), ('wooden_slab', 1, 0)]


In [ ]:
tech_tree["wood"]